In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 1. Hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. Data Preprocessing: No resizing, keep original 32x32 size
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# 3. Load CIFAR-10 Dataset
full_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#full_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 4. Filter Dataset for Cats and Dogs
def filter_cats_dogs(dataset):
    targets = torch.tensor(dataset.targets)  # Convert labels to a tensor
    mask = (targets == 3) | (targets == 5)  # Keep only labels 3 (cat) and 5 (dog)
    dataset.targets = targets[mask].tolist()  # Update targets
    dataset.data = dataset.data[mask.numpy()]  # Update data
    return dataset

train_dataset = filter_cats_dogs(full_train_dataset)
#test_dataset = filter_cats_dogs(full_test_dataset)

# 5. Update Labels: Map [3, 5] -> [0, 1]
def remap_labels(dataset):
    dataset.targets = [0 if label == 3 else 1 for label in dataset.targets]  # Cat = 0, Dog = 1
    return dataset

train_dataset = remap_labels(train_dataset)
#test_dataset = remap_labels(test_dataset)

# 6. Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# RPQ Function
def rpq(images):
    #sign_quantization
    def quantize(signature):
        for i in range(0,columns) :
            if signature[i] < 0:
                signature[i] = 1
            else:
                signature[i] = 0
        return signature

    rows = 1024
    columns = 20
    random_rpq_matrix = torch.randn(rows, columns, device=device)  # Move RPQ matrix to GPU

    # Flatten 32 * 32 to 1 * 1024
    flattened_image = images.view(-1)

    # Dot product of input vector and R
    signature = torch.matmul(flattened_image, random_rpq_matrix)

    # Quantization -> sign-based
    signature_quantized = quantize(signature)
    return signature_quantized

# MCache
mcache = {}
cache_hits = 0
cache_misses = 0
binary_key = ''

# 7. Define a Simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog

    def forward(self, x):
        global cache_hits, binary_key, cache_misses
        if binary_key in mcache:
            # Already first layer output is available
            cache_hits += 1
            x = mcache[binary_key].detach()
        else:
            cache_misses += 1
            x = self.pool(torch.relu(self.conv1(x)))
            mcache[binary_key] = x.detach()
            
        # Second layer is continued normally now
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)  # Move model to GPU

# 8. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 9. Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move images and labels to GPU

        # Generate RPQ signature and binary key
        rpq_signature_output = rpq(images)

        # Convert the binary tensor to a string representation of binary
        binary_key = ''.join(map(str, rpq_signature_output.tolist()))  # tensor to list and then to string

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")
print("Cache_hits :", cache_hits)
print("Cache_misses:", cache_misses)

Using device: cuda
Files already downloaded and verified
Epoch 1/10
Epoch 1/10, Loss: 0.6674
Epoch 2/10
Epoch 2/10, Loss: 0.6357
Epoch 3/10
Epoch 3/10, Loss: 0.6087
Epoch 4/10
Epoch 4/10, Loss: 0.5826
Epoch 5/10
Epoch 5/10, Loss: 0.5595
Epoch 6/10
Epoch 6/10, Loss: 0.5359
Epoch 7/10
Epoch 7/10, Loss: 0.5089
Epoch 8/10
Epoch 8/10, Loss: 0.4736
Epoch 9/10
Epoch 9/10, Loss: 0.4476
Epoch 10/10
Epoch 10/10, Loss: 0.4232
Training complete!
Cache_hits : 4619
Cache_misses: 95381


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 1. Hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. Data Preprocessing: No resizing, keep original 32x32 size
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# 3. Load CIFAR-10 Dataset
full_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#full_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 4. Filter Dataset for Cats and Dogs
def filter_cats_dogs(dataset):
    targets = torch.tensor(dataset.targets)  # Convert labels to a tensor
    mask = (targets == 3) | (targets == 5)  # Keep only labels 3 (cat) and 5 (dog)
    dataset.targets = targets[mask].tolist()  # Update targets
    dataset.data = dataset.data[mask.numpy()]  # Update data
    return dataset

train_dataset = filter_cats_dogs(full_train_dataset)
#test_dataset = filter_cats_dogs(full_test_dataset)

# 5. Update Labels: Map [3, 5] -> [0, 1]
def remap_labels(dataset):
    dataset.targets = [0 if label == 3 else 1 for label in dataset.targets]  # Cat = 0, Dog = 1
    return dataset

train_dataset = remap_labels(train_dataset)
#test_dataset = remap_labels(test_dataset)

# 6. Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# RPQ Function
def rpq(images):
    #sign_quantization
    def quantize(signature):
        for i in range(0,columns) :
            if signature[i] < 0:
                signature[i] = 1
            else:
                signature[i] = 0
        return signature

    rows = 1024
    columns = 20
    random_rpq_matrix = torch.randn(rows, columns, device=device)  # Move RPQ matrix to GPU

    # Flatten 32 * 32 to 1 * 1024
    flattened_image = images.view(-1)

    # Dot product of input vector and R
    signature = torch.matmul(flattened_image, random_rpq_matrix)

    # Quantization -> sign-based
    signature_quantized = quantize(signature)
    return signature_quantized

# MCache
mcache = {}
cache_hits = 0
cache_misses = 0
binary_key = ''

# 7. Define a Simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog

    def forward(self, x):
        global cache_hits, binary_key, cache_misses
        if binary_key in mcache:
            # Already first layer output is available
            cache_hits += 1
            x = mcache[binary_key].detach()
        else:
            cache_misses += 1
            x = self.pool(torch.relu(self.conv1(x)))
            mcache[binary_key] = x.detach()
            
        # Second layer is continued normally now
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)  # Move model to GPU

# 8. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 9. Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move images and labels to GPU

        # Generate RPQ signature and binary key
        rpq_signature_output = rpq(images)

        # Convert the binary tensor to a string representation of binary
        binary_key = ''.join(map(str, rpq_signature_output.tolist()))  # tensor to list and then to string

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")
print("Cache_hits :", cache_hits)
print("Cache_misses:", cache_misses)

In [ ]:
# 10. Test the Model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")